In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [ ]:

import os
import matplotlib.pyplot as plt

import networkx as nx
from IPython.core.display import SVG, display
import pandas as pd 

from autorocks.envs.synthetic.funcs import levy

_DIM = 25
param_space = levy.make_levy_space(_DIM)
obj_name = "target"

iter_count = {25: 100, 50: 125, 100: 200, 300: 500}[_DIM]

In [ ]:
from autorocks.envs.postgres.benchmarks.benchbase import BenchmarkClass
from autorocks.data.loader.all_models_result_aggregator import create_all_models_comparison_dataset
import autorocks.dir_struct as DIR

exp_dir = DIR.PackageRootDir / "ProcessedDataNew" / f"Levy{_DIM}D/{obj_name}/{len(param_space)}_params/{iter_count}_iter"
model_comparison_data = create_all_models_comparison_dataset(exp_dir)


In [ ]:
df = model_comparison_data.combine_sys_params_metric().copy().drop(columns=['iteration', 'step'])

df_fit = df[df['model'] == "Random"].copy().drop(columns=['model'])
df_test = df[df['model'] != "Random"].copy().drop(columns=['model'])

In [ ]:
import bnlearn as bn

golden_dag = levy.make_struct(_DIM)
# Discretize the continous columns
df_disc = bn.discretize(df_fit, list(golden_dag.edges), df_fit.columns, max_iterations=8)
df_test_disc = bn.discretize(df_test, list(golden_dag.edges), df_test.columns, max_iterations=1)


In [ ]:
from pgmpy import metrics 

bn_model = bn.parameter_learning.fit(bn.make_DAG(list(golden_dag.edges)), df_disc.copy(), verbose = 4)
golden_metric = metrics.structure_score(bn_model['model'], df_test_disc, scoring_method="bic")


In [ ]:
# Baseline DAG = parameters to target - no structure 
baseline_dag = bn.make_DAG([(p, 'target') for p in param_space])
baseline_dag_disc = bn.discretize(df_fit, list(golden_dag.edges), df_fit.columns, max_iterations=8)
baseline_dag_bn = bn.parameter_learning.fit(baseline_dag, df_disc.copy(), verbose = 4)
no_structure_score = baseline_dag_bn['structure_scores']

In [ ]:

baseline_metric = metrics.structure_score(baseline_dag_bn['model'], df_test[list(baseline_dag_bn['model'].nodes)], scoring_method="bic")


In [ ]:
baseline_metric / golden_metric 

In [ ]:
intermediates = []
for node in G.nodes:
    if node not in param_space and node != "target":
        intermediates.append(node)


In [ ]:
baseline_model = baseline_dag_bn['model']
baseline_model.add_nodes_from(intermediates)
baseline_model
# metrics.structure_score(baseline_dag['model'], df_disc, scoring_method="bic")

In [ ]:

metrics.structure_score(baseline_model, df_disc, scoring_method="bic")
